In [52]:
import sympy as sp

# Dephasing channel
t, T2, omega = sp.symbols('t T2 w', real=True, positive=True)
gamma = sp.exp(t / T2) 
rot_plus = sp.exp(sp.I * omega * t)
rot_minus = sp.exp(-sp.I * omega * t)
R_liouville = sp.diag(1, gamma*rot_plus, gamma*rot_minus, 1)

In [81]:
from qiskit.quantum_info import Operator
import numpy as np

a = np.pi/4

Z = np.array([[1, 0],
              [0, -1]])

Rz = np.array([[np.exp(-1j * a/ 2), 0],
              [0, np.exp(1j * a/ 2)]])

rho = 0.5 * np.array([[1,1], [1,1]])

In [83]:
Rz @ Z @ rho @ Z @ Rz.conj().T

array([[ 0.5       +0.j        , -0.35355339+0.35355339j],
       [-0.35355339-0.35355339j,  0.5       +0.j        ]])

In [53]:
# Basis change matrix (Pauli -> Liouville basis)
B = sp.Matrix([
    [1, 0, 0, 1],
    [0, 1, sp.I, 0],
    [0, 1,  -sp.I, 0],
    [1, 0, 0, -1],
]) / sp.sqrt(2) 

# Basis change matrix (Liouville -> Pauli basis)
B_inv = B.inv()

# Pauli transfer matrix: R = B⁻¹ * E * B
R_pauli = B_inv * R_liouville * B
R_pauli = sp.simplify(R_pauli)

E00 = sp.Matrix([1,0,0,0])
E10 = sp.Matrix([0,1,0,0])
E01 = sp.Matrix([0,0,1,0])
E11 = sp.Matrix([0,0,0,1])
vecs = [E00, E10, E01, E11]

def vec_to_matrix(v):
    return sp.Matrix([[v[0], v[2]], [v[1], v[3]]])


choi = sp.zeros(4, 4)
for i in range(2):
    for j in range(2):
        E_ij = vecs[i + 2*j]
        R_liouville = B * R_pauli * B_inv
        transformed_vec = R_liouville * E_ij
        transformed_matrix = vec_to_matrix(transformed_vec)
        
        for k in range(2):
            for l in range(2):
                choi[2*i + k, 2*j + l] = transformed_matrix[k, l]

choi = sp.simplify(choi)
sp.pprint(choi)

⎡                    t⋅(-ⅈ⋅T₂⋅w + 1)⎤
⎢                    ───────────────⎥
⎢                          T₂       ⎥
⎢     1       0  0  ℯ               ⎥
⎢                                   ⎥
⎢     0       0  0         0        ⎥
⎢                                   ⎥
⎢     0       0  0         0        ⎥
⎢                                   ⎥
⎢         t                         ⎥
⎢ ⅈ⋅t⋅w + ──                        ⎥
⎢         T₂                        ⎥
⎣ℯ            0  0         1        ⎦


In [58]:
V, D = choi.diagonalize()  # A = P * D * P⁻¹
V_dag = V.H

def construct_C(sign=1):
    C = sp.zeros(*choi.shape)
    for i in range(D.shape[0]):
        lam = D[i, i]
        vi = V[:, i]
        norm = sp.sqrt((vi.H * vi)[0])
        vi_norm = vi / norm
        proj = vi_norm * vi_norm.H
        weight = sp.Max(0, sign * lam)
        C += weight * proj
    return sp.simplify(C)

C_plus = construct_C(+1)
C_minus = construct_C(-1)

print("C_plus =")
sp.pprint(C_plus)

print("\nC_minus =")
sp.pprint(C_minus)

C_plus =
⎡     t                                          ⎛ t                             ⎞        ⎤
⎢     ──                                         ⎜ ──                            ⎟        ⎥
⎢     T₂                                         ⎜ T₂                            ⎟  -ⅈ⋅t⋅w⎥
⎢    ℯ     Max(0, 1 - exp(t/T2))   1             ⎝ℯ   - Max(0, 1 - exp(t/T2)) + 1⎠⋅ℯ      ⎥
⎢    ─── + ───────────────────── + ─       0  0  ─────────────────────────────────────────⎥
⎢     2              2             2                                 2                    ⎥
⎢                                                                                         ⎥
⎢                   0                      0  0                      0                    ⎥
⎢                                                                                         ⎥
⎢                   0                      0  0                      0                    ⎥
⎢                                                                      

In [57]:
V

Matrix([
[0, 0, -exp(-I*t*w), exp(-I*t*w)],
[1, 0,            0,           0],
[0, 1,            0,           0],
[0, 0,            1,           1]])

In [55]:
P

Matrix([
[0, 0, -exp(-I*t*w), exp(-I*t*w)],
[1, 0,            0,           0],
[0, 1,            0,           0],
[0, 0,            1,           1]])

In [24]:
# Pauli matrices
I = sp.Matrix([[1, 0], [0, 1]])
X = sp.Matrix([[0, 1], [1, 0]])
Y = sp.Matrix([[0, -sp.I], [sp.I, 0]])
Z = sp.Matrix([[1, 0], [0, -1]])
paulis = [I, X, Y, Z]

I = sp.Matrix([[1,0,0,1]])
X = sp.Matrix([[0,1,1,0]])
Y = sp.Matrix([[0,sp.I,-sp.I,0]])
Z = sp.Matrix([[1,0,0,-1]])

In [27]:
E00 = sp.Matrix([1,0,0,0])
E10 = sp.Matrix([0,1,0,0])
E01 = sp.Matrix([0,0,1,0])
E11 = sp.Matrix([0,0,0,1])

B * R_pauli * B_inv * E00

In [34]:
B * R_pauli * B_inv * E00

Matrix([
[1],
[0],
[0],
[0]])